In [ ]:
import alpaca_trade_api as tradeapi
import threading
from time import sleep 
import json
import logging
import time
import datetime

#https://github.com/alpacahq/alpaca-trade-api-python/blob/master/examples/long-short.py
    
#authentication and connection details
APCA_API_KEY_ID = 'PKPIODDUG9GGMDMMDBV9'
APCA_API_SECRET_KEY = '4Tjm3yyvI8xAP4Shp9jeS5rO6iQ59dPI9jbaIkxo'
APCA_API_BASE_URL = 'https://paper-api.alpaca.markets'

class LongShort:
  def __init__(self):
    self.alpaca = tradeapi.REST(APCA_API_KEY_ID, APCA_API_SECRET_KEY, APCA_API_BASE_URL, api_version='v2')

    stockUniverse = ['DOMO', 'TLRY', 'SQ', 'MRO', 'AAPL', 'GM', 'SNAP', 'SHOP', 'SPLK', 'BA', 'AMZN', 'SUI', 'SUN', 'TSLA', 'CGC', 'SPWR', 'NIO', 'CAT', 'MSFT', 'PANW', 'OKTA', 'TWTR', 'TM', 'RTN', 'ATVI', 'GS', 'BAC', 'MS', 'TWLO', 'QCOM', ]
    # Format the allStocks variable for use in the class.
    self.allStocks = []
    for stock in stockUniverse:
      self.allStocks.append([stock, 0])

    self.long = []
    self.short = []
    self.qShort = None
    self.qLong = None
    self.adjustedQLong = None
    self.adjustedQShort = None
    self.blacklist = set()
    self.longAmount = 0
    self.shortAmount = 0
    self.timeToClose = None

  def run(self):
    # First, cancel any existing orders so they don't impact our buying power.
    orders = self.alpaca.list_orders(status="open")
    for order in orders:
      self.alpaca.cancel_order(order.id)

    # Wait for market to open.
    print("Waiting for market to open...")
    tAMO = threading.Thread(target=self.awaitMarketOpen)
    tAMO.start()
    tAMO.join()
    print("Market opened.")

    # Rebalance the portfolio every minute, making necessary trades.
    while True:

      # Figure out when the market will close so we can prepare to sell beforehand.
      clock = self.alpaca.get_clock()
      closingTime = clock.next_close.replace(tzinfo=datetime.timezone.utc).timestamp()
      currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
      self.timeToClose = closingTime - currTime

      if(self.timeToClose < (60 * 15)):
        # Close all positions when 15 minutes til market close.
        print("Market closing soon.  Closing positions.")

        positions = self.alpaca.list_positions()
        for position in positions:
          if(position.side == 'long'):
            orderSide = 'sell'
          else:
            orderSide = 'buy'
          qty = abs(int(float(position.qty)))
          respSO = []
          tSubmitOrder = threading.Thread(target=self.submitOrder(qty, position.symbol, orderSide, respSO))
          tSubmitOrder.start()
          tSubmitOrder.join()

        # Run script again after market close for next trading day.
        print("Sleeping until market close (15 minutes).")
        time.sleep(60 * 15)
      else:
        # Rebalance the portfolio.
        tRebalance = threading.Thread(target=self.rebalance)
        tRebalance.start()
        tRebalance.join()
        time.sleep(60)

  # Wait for market to open.
  def awaitMarketOpen(self):
    isOpen = self.alpaca.get_clock().is_open
    while(not isOpen):
      clock = self.alpaca.get_clock()
      openingTime = clock.next_open.replace(tzinfo=datetime.timezone.utc).timestamp()
      currTime = clock.timestamp.replace(tzinfo=datetime.timezone.utc).timestamp()
      timeToOpen = int((openingTime - currTime) / 60)
      print(str(timeToOpen) + " minutes til market open.")
      time.sleep(60)
      isOpen = self.alpaca.get_clock().is_open

  def rebalance(self):
    tRerank = threading.Thread(target=self.rerank)
    tRerank.start()
    tRerank.join()

    # Clear existing orders again.
    orders = self.alpaca.list_orders(status="open")
    for order in orders:
      self.alpaca.cancel_order(order.id)

    print("We are taking a long position in: " + str(self.long))
    print("We are taking a short position in: " + str(self.short))
    # Remove positions that are no longer in the short or long list, and make a list of positions that do not need to change.  Adjust position quantities if needed.
    executed = [[], []]
    positions = self.alpaca.list_positions()
    self.blacklist.clear()
    for position in positions:
      if(self.long.count(position.symbol) == 0):
        # Position is not in long list.
        if(self.short.count(position.symbol) == 0):
          # Position not in short list either.  Clear position.
          if(position.side == "long"):
            side = "sell"
          else:
            side = "buy"
          respSO = []
          tSO = threading.Thread(target=self.submitOrder, args=[abs(int(float(position.qty))), position.symbol, side, respSO])
          tSO.start()
          tSO.join()
        else:
          # Position in short list.
          if(position.side == "long"):
            # Position changed from long to short.  Clear long position to prepare for short position.
            side = "sell"
            respSO = []
            tSO = threading.Thread(target=self.submitOrder, args=[int(float(position.qty)), position.symbol, side, respSO])
            tSO.start()
            tSO.join()
          else:
            if(abs(int(float(position.qty))) == self.qShort):
              # Position is where we want it.  Pass for now.
              pass
            else:
              # Need to adjust position amount
              diff = abs(int(float(position.qty))) - self.qShort
              if(diff > 0):
                # Too many short positions.  Buy some back to rebalance.
                side = "buy"
              else:
                # Too little short positions.  Sell some more.
                side = "sell"
              respSO = []
              tSO = threading.Thread(target=self.submitOrder, args=[abs(diff), position.symbol, side, respSO])
              tSO.start()
              tSO.join()
            executed[1].append(position.symbol)
            self.blacklist.add(position.symbol)
      else:
        # Position in long list.
        if(position.side == "short"):
          # Position changed from short to long.  Clear short position to prepare for long position.
          respSO = []
          tSO = threading.Thread(target=self.submitOrder, args=[abs(int(float(position.qty))), position.symbol, "buy", respSO])
          tSO.start()
          tSO.join()
        else:
          if(int(float(position.qty)) == self.qLong):
            # Position is where we want it.  Pass for now.
            pass
          else:
            # Need to adjust position amount.
            diff = abs(int(float(position.qty))) - self.qLong
            if(diff > 0):
              # Too many long positions.  Sell some to rebalance.
              side = "sell"
            else:
              # Too little long positions.  Buy some more.
              side = "buy"
            respSO = []
            tSO = threading.Thread(target=self.submitOrder, args=[abs(diff), position.symbol, side, respSO])
            tSO.start()
            tSO.join()
          executed[0].append(position.symbol)
          self.blacklist.add(position.symbol)

    # Send orders to all remaining stocks in the long and short list.
    respSendBOLong = []
    tSendBOLong = threading.Thread(target=self.sendBatchOrder, args=[self.qLong, self.long, "buy", respSendBOLong])
    tSendBOLong.start()
    tSendBOLong.join()
    respSendBOLong[0][0] += executed[0]
    if(len(respSendBOLong[0][1]) > 0):
      # Handle rejected/incomplete orders and determine new quantities to purchase.
      respGetTPLong = []
      tGetTPLong = threading.Thread(target=self.getTotalPrice, args=[respSendBOLong[0][0], respGetTPLong])
      tGetTPLong.start()
      tGetTPLong.join()
      if (respGetTPLong[0] > 0):
        self.adjustedQLong = self.longAmount // respGetTPLong[0]
      else:
        self.adjustedQLong = -1
    else:
      self.adjustedQLong = -1

    respSendBOShort = []
    tSendBOShort = threading.Thread(target=self.sendBatchOrder, args=[self.qShort, self.short, "sell", respSendBOShort])
    tSendBOShort.start()
    tSendBOShort.join()
    respSendBOShort[0][0] += executed[1]
    if(len(respSendBOShort[0][1]) > 0):
      # Handle rejected/incomplete orders and determine new quantities to purchase.
      respGetTPShort = []
      tGetTPShort = threading.Thread(target=self.getTotalPrice, args=[respSendBOShort[0][0], respGetTPShort])
      tGetTPShort.start()
      tGetTPShort.join()
      if(respGetTPShort[0] > 0):
        self.adjustedQShort = self.shortAmount // respGetTPShort[0]
      else:
        self.adjustedQShort = -1
    else:
      self.adjustedQShort = -1

    # Reorder stocks that didn't throw an error so that the equity quota is reached.
    if(self.adjustedQLong > -1):
      self.qLong = int(self.adjustedQLong - self.qLong)
      for stock in respSendBOLong[0][0]:
        respResendBOLong = []
        tResendBOLong = threading.Thread(target=self.submitOrder, args=[self.qLong, stock, "buy", respResendBOLong])
        tResendBOLong.start()
        tResendBOLong.join()

    if(self.adjustedQShort > -1):
      self.qShort = int(self.adjustedQShort - self.qShort)
      for stock in respSendBOShort[0][0]:
        respResendBOShort = []
        tResendBOShort = threading.Thread(target=self.submitOrder, args=[self.qShort, stock, "sell", respResendBOShort])
        tResendBOShort.start()
        tResendBOShort.join()

  # Re-rank all stocks to adjust longs and shorts.
  def rerank(self):
    tRank = threading.Thread(target=self.rank)
    tRank.start()
    tRank.join()

    # Grabs the top and bottom quarter of the sorted stock list to get the long and short lists.
    longShortAmount = len(self.allStocks) // 4
    self.long = []
    self.short = []
    for i, stockField in enumerate(self.allStocks):
      if(i < longShortAmount):
        self.short.append(stockField[0])
      elif(i > (len(self.allStocks) - 1 - longShortAmount)):
        self.long.append(stockField[0])
      else:
        continue

    # Determine amount to long/short based on total stock price of each bucket.
    equity = int(float(self.alpaca.get_account().equity))

    self.shortAmount = equity * 0.30
    self.longAmount = equity + self.shortAmount

    respGetTPLong = []
    tGetTPLong = threading.Thread(target=self.getTotalPrice, args=[self.long, respGetTPLong])
    tGetTPLong.start()
    tGetTPLong.join()

    respGetTPShort = []
    tGetTPShort = threading.Thread(target=self.getTotalPrice, args=[self.short, respGetTPShort])
    tGetTPShort.start()
    tGetTPShort.join()

    self.qLong = int(self.longAmount // respGetTPLong[0])
    self.qShort = int(self.shortAmount // respGetTPShort[0])

  # Get the total price of the array of input stocks.
  def getTotalPrice(self, stocks, resp):
    totalPrice = 0
    for stock in stocks:
      bars = self.alpaca.get_barset(stock, "minute", 1)
      totalPrice += bars[stock][0].c
    resp.append(totalPrice)

  # Submit a batch order that returns completed and uncompleted orders.
  def sendBatchOrder(self, qty, stocks, side, resp):
    executed = []
    incomplete = []
    for stock in stocks:
      if(self.blacklist.isdisjoint({stock})):
        respSO = []
        tSubmitOrder = threading.Thread(target=self.submitOrder, args=[qty, stock, side, respSO])
        tSubmitOrder.start()
        tSubmitOrder.join()
        if(not respSO[0]):
          # Stock order did not go through, add it to incomplete.
          incomplete.append(stock)
        else:
          executed.append(stock)
        respSO.clear()
    resp.append([executed, incomplete])

  # Submit an order if quantity is above 0.
  def submitOrder(self, qty, stock, side, resp):
    if(qty > 0):
      try:
        self.alpaca.submit_order(stock, qty, side, "market", "day")
        print("Market order of | " + str(qty) + " " + stock + " " + side + " | completed.")
        resp.append(True)
      except:
        print("Order of | " + str(qty) + " " + stock + " " + side + " | did not go through.")
        resp.append(False)
    else:
      print("Quantity is 0, order of | " + str(qty) + " " + stock + " " + side + " | not completed.")
      resp.append(True)

  # Get percent changes of the stock prices over the past 10 minutes.
  def getPercentChanges(self):
    length = 10
    for i, stock in enumerate(self.allStocks):
      bars = self.alpaca.get_barset(stock[0], 'minute', length)
      self.allStocks[i][1] = (bars[stock[0]][len(bars[stock[0]]) - 1].c - bars[stock[0]][0].o) / bars[stock[0]][0].o

  # Mechanism used to rank the stocks, the basis of the Long-Short Equity Strategy.
  def rank(self):
    # Ranks all stocks by percent change over the past 10 minutes (higher is better).
    tGetPC = threading.Thread(target=self.getPercentChanges)
    tGetPC.start()
    tGetPC.join()

    # Sort the stocks in place by the percent change field (marked by pc).
    self.allStocks.sort(key=lambda x: x[1])

# Run the LongShort class
ls = LongShort()
ls.run()

Waiting for market to open...
Market opened.
We are taking a long position in: ['SQ', 'TWLO', 'TM', 'SPWR', 'AMZN', 'TSLA', 'SUI']
We are taking a short position in: ['GS', 'MRO', 'GM', 'NIO', 'TLRY', 'RTN', 'BA']
Market order of | 11 SPCE sell | completed.
Market order of | 1 PETS sell | completed.
Market order of | 29 SQ buy | completed.
Market order of | 29 TWLO buy | completed.
Market order of | 29 TM buy | completed.
Market order of | 29 SPWR buy | completed.
Market order of | 29 AMZN buy | completed.
Market order of | 29 TSLA buy | completed.
Market order of | 29 SUI buy | completed.
Market order of | 55 GS sell | completed.
Market order of | 55 MRO sell | completed.
Market order of | 55 GM sell | completed.
Market order of | 55 NIO sell | completed.
Order of | 55 TLRY sell | did not go through.
Order of | 55 RTN sell | did not go through.
Market order of | 55 BA sell | completed.
Market order of | 18 GS sell | completed.
Market order of | 18 MRO sell | completed.
Market order of

Market order of | 48 SQ buy | completed.
Market order of | 104 SPWR sell | completed.
Market order of | 48 SPLK buy | completed.
Market order of | 48 SHOP buy | completed.
Market order of | 48 QCOM buy | completed.
Market order of | 48 NIO buy | completed.
Market order of | 104 MS sell | completed.
Market order of | 104 MRO sell | completed.
Market order of | 97 GS buy | completed.
Market order of | 97 GM buy | completed.
Market order of | 104 BAC sell | completed.
Market order of | 30 AMZN sell | completed.
Market order of | 78 TWLO buy | completed.
Order of | 201 TLRY sell | did not go through.
Market order of | 201 TWTR sell | completed.
Market order of | 201 SNAP sell | completed.
Market order of | 12 TWTR sell | completed.
Market order of | 12 SNAP sell | completed.
Market order of | 12 SPWR sell | completed.
Market order of | 12 MS sell | completed.
Market order of | 12 MRO sell | completed.
Market order of | 12 BAC sell | completed.
We are taking a long position in: ['TWLO', 'SU

Market order of | 40 DOMO sell | completed.
Market order of | 40 BAC sell | completed.
Market order of | 251 ATVI sell | completed.
Market order of | 211 BA buy | completed.
Order of | 56 TLRY sell | did not go through.
Order of | 56 CGC sell | did not go through.
Order of | 56 SUN sell | did not go through.
Market order of | 56 SUI sell | completed.
Market order of | 6 SUI sell | completed.
Market order of | 6 SQ sell | completed.
Market order of | 6 SNAP sell | completed.
Market order of | 6 OKTA sell | completed.
We are taking a long position in: ['TWTR', 'SNAP', 'ATVI', 'BAC', 'TWLO', 'DOMO', 'TM']
We are taking a short position in: ['TLRY', 'CGC', 'SUN', 'NIO', 'SQ', 'OKTA', 'MS']
Market order of | 28 TWTR buy | completed.
Market order of | 28 TM buy | completed.
Market order of | 62 SUI buy | completed.
Market order of | 9 SQ sell | completed.
Market order of | 211 SPLK sell | completed.
Market order of | 62 SNAP buy | completed.
Market order of | 9 OKTA sell | completed.
Market 

Market order of | 24 SUI sell | completed.
Market order of | 24 SPLK sell | completed.
Market order of | 24 SNAP sell | completed.
Market order of | 24 NIO sell | completed.
We are taking a long position in: ['AMZN', 'PANW', 'TLRY', 'OKTA', 'TM', 'SQ', 'TWLO']
We are taking a short position in: ['SUN', 'CGC', 'SUI', 'SNAP', 'RTN', 'TSLA', 'DOMO']
Market order of | 1 TWLO buy | completed.
Market order of | 1 TM buy | completed.
Market order of | 59 SUI buy | completed.
Market order of | 1 SQ buy | completed.
Market order of | 80 SPLK buy | completed.
Market order of | 1 SPCE sell | completed.
Market order of | 59 SNAP buy | completed.
Market order of | 1 PANW buy | completed.
Market order of | 1 OKTA buy | completed.
Market order of | 80 NIO buy | completed.
Market order of | 33 BA sell | completed.
Market order of | 1 AMZN buy | completed.
Market order of | 34 TLRY buy | completed.
Order of | 21 SUN sell | did not go through.
Order of | 21 CGC sell | did not go through.
Order of | 21 R

Market order of | 6 SUI buy | completed.
Market order of | 19 SQ sell | completed.
Market order of | 6 SPWR buy | completed.
Market order of | 19 SPLK sell | completed.
Market order of | 19 SHOP sell | completed.
Market order of | 19 PANW sell | completed.
Market order of | 47 OKTA sell | completed.
Market order of | 6 NIO buy | completed.
Market order of | 19 MRO sell | completed.
Market order of | 6 GS buy | completed.
Market order of | 6 DOMO buy | completed.
Market order of | 28 AMZN buy | completed.
Market order of | 28 BA buy | completed.
Order of | 70 SUN sell | did not go through.
Order of | 70 TLRY sell | did not go through.
Market order of | 6 SUI sell | completed.
Market order of | 6 SPWR sell | completed.
Market order of | 6 NIO sell | completed.
Market order of | 6 GS sell | completed.
Market order of | 6 DOMO sell | completed.
We are taking a long position in: ['AMZN', 'PANW', 'SQ', 'ATVI', 'SHOP', 'BA', 'SPLK']
We are taking a short position in: ['DOMO', 'SUI', 'SPWR', '

Market order of | 17 SQ buy | completed.
Market order of | 1 SPLK buy | completed.
Market order of | 1 SHOP buy | completed.
Market order of | 163 QCOM sell | completed.
Market order of | 17 OKTA buy | completed.
Market order of | 245 NIO sell | completed.
Market order of | 163 MSFT sell | completed.
Market order of | 163 MRO sell | completed.
Market order of | 245 GM sell | completed.
Market order of | 1 DOMO buy | completed.
Market order of | 163 CGC sell | completed.
Market order of | 245 BA sell | completed.
Market order of | 82 TSLA buy | completed.
Market order of | 82 GS buy | completed.
Market order of | 82 TWTR buy | completed.
Market order of | 16 SUI sell | completed.
Order of | 16 SUN sell | did not go through.
Market order of | 16 PANW sell | completed.
Quantity is 0, order of | 0 SUI sell | not completed.
Quantity is 0, order of | 0 PANW sell | not completed.
Quantity is 0, order of | 0 TWLO sell | not completed.
Quantity is 0, order of | 0 SPLK sell | not completed.
Quan

Market order of | 42 TM sell | completed.
Market order of | 42 SPWR sell | completed.
Market order of | 76 SPLK sell | completed.
Market order of | 76 SHOP sell | completed.
Market order of | 76 QCOM sell | completed.
Market order of | 41 OKTA sell | completed.
Market order of | 42 NIO sell | completed.
Market order of | 76 MRO sell | completed.
Market order of | 76 GS sell | completed.
Market order of | 41 DOMO sell | completed.
Market order of | 42 BA sell | completed.
Market order of | 35 AMZN buy | completed.
Market order of | 35 AAPL buy | completed.
Market order of | 35 CAT buy | completed.
Market order of | 35 SUN buy | completed.
Market order of | 35 TLRY buy | completed.
Market order of | 60 SNAP sell | completed.
Market order of | 60 SQ sell | completed.
We are taking a long position in: ['SUN', 'OKTA', 'SUI', 'AAPL', 'DOMO', 'TLRY', 'CGC']
We are taking a short position in: ['SPWR', 'NIO', 'SQ', 'TM', 'TWLO', 'TWTR', 'MRO']
Market order of | 3 TWTR buy | completed.
Market or

Order of | 15 SUN sell | did not go through.
Order of | 15 RTN sell | did not go through.
Market order of | 15 SPLK sell | completed.
Market order of | 2 SPLK sell | completed.
Market order of | 2 TWLO sell | completed.
Market order of | 2 TM sell | completed.
Market order of | 2 SHOP sell | completed.
Market order of | 2 OKTA sell | completed.
We are taking a long position in: ['MSFT', 'SPWR', 'TWTR', 'SQ', 'CGC', 'NIO', 'BA']
We are taking a short position in: ['SUN', 'TM', 'OKTA', 'RTN', 'SHOP', 'TWLO', 'PANW']
Market order of | 193 TWTR buy | completed.
Market order of | 2 TWLO buy | completed.
Market order of | 2 TM buy | completed.
Market order of | 37 TLRY sell | completed.
Market order of | 193 SQ buy | completed.
Market order of | 17 SPLK buy | completed.
Market order of | 2 SHOP buy | completed.
Market order of | 2 OKTA buy | completed.
Market order of | 193 MSFT buy | completed.
Market order of | 193 CGC buy | completed.
Market order of | 193 BA buy | completed.
Market order

Market order of | 12 TSLA sell | completed.
Market order of | 1 SHOP sell | completed.
Market order of | 1 TSLA sell | completed.
Market order of | 1 TWTR sell | completed.
Market order of | 1 TM sell | completed.
Market order of | 1 SNAP sell | completed.
We are taking a long position in: ['GM', 'SPWR', 'BA', 'SPLK', 'NIO', 'CGC', 'TWLO']
We are taking a short position in: ['SUN', 'SHOP', 'TM', 'SNAP', 'RTN', 'SQ', 'TWTR']
Market order of | 8 TWTR sell | completed.
Market order of | 13 TSLA buy | completed.
Market order of | 8 TM sell | completed.
Market order of | 158 SUI sell | completed.
Market order of | 34 SPLK buy | completed.
Market order of | 8 SNAP sell | completed.
Market order of | 8 SHOP sell | completed.
Market order of | 158 MSFT sell | completed.
Market order of | 34 GM buy | completed.
Market order of | 158 DOMO sell | completed.
Market order of | 34 CGC buy | completed.
Market order of | 34 BA buy | completed.
Market order of | 192 SPWR buy | completed.
Market order o

Market order of | 71 MRO sell | completed.
Market order of | 71 SNAP sell | completed.
Market order of | 71 TM sell | completed.
Market order of | 71 GM sell | completed.
We are taking a long position in: ['ATVI', 'TSLA', 'NIO', 'SNAP', 'SPWR', 'DOMO', 'TWLO']
We are taking a short position in: ['SUN', 'TM', 'RTN', 'GM', 'TLRY', 'MS', 'CAT']
Market order of | 10 TSLA buy | completed.
Market order of | 96 TM buy | completed.
Market order of | 10 SPWR buy | completed.
Market order of | 81 SPLK sell | completed.
Market order of | 155 SNAP buy | completed.
Market order of | 81 OKTA sell | completed.
Market order of | 10 NIO buy | completed.
Market order of | 155 MRO buy | completed.
Market order of | 96 GM buy | completed.
Market order of | 10 DOMO buy | completed.
Market order of | 10 ATVI buy | completed.
Market order of | 91 SNAP buy | completed.
Market order of | 91 TWLO buy | completed.
Order of | 59 SUN sell | did not go through.
Order of | 59 RTN sell | did not go through.
Order of 